##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras 前処理レイヤーを使って構造化データを分類する

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">    TensorFlow.org で表示</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/structured_data/preprocessing_layers.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">    Google Colab で実行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/structured_data/preprocessing_layers.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">    GitHub でソースを表示</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/structured_data/preprocessing_layers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

このチュートリアルでは、構造化データ（CSV のタブ区切りデータ）を分類する方法を実演します。モデルの定義には [Keras](https://www.tensorflow.org/guide/keras) を使用し、CSV の列からモデルのトレーニングに使用する特徴量にマッピングするための懸け橋として[前処理レイヤー](https://www.tensorflow.org/guide/keras/preprocessing_layers)を使用します。このチュートリアルに含まれるコードは、次のことを行います。

- [Pandas](https://pandas.pydata.org/) を使って CSV ファイルを読み込みます。
- [tf.data](https://www.tensorflow.org/guide/datasets) を使用して、行をバッチ化してシャッフルする入力パイプラインを構築します。
- Keras 前処理レイヤーを使ってモデルをトレーニングするために使用する特徴量に、CSV のカラムをマッピングします。
- Keras を使用して、モデルを構築、トレーニング、および評価します。

注意: このチュートリアルは、「[特徴量カラムを使って構造化データを分類する](https://www.tensorflow.org/tutorials/structured_data/feature_columns)」に類似しています。このバージョンでは、新しい実験的 Keras [前処理レイヤー](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing)を使用しており、`tf.feature_column` を使っていません。Keras 前処理レイヤーはより直感的であり、デプロイを単純化できるようにモデル内に簡単に含めることができます。

## Dataset

PetFinder [データセット](https://www.kaggle.com/c/petfinder-adoption-prediction)の簡易バージョンを使用します。CSV には数千行のデータが含まれており各行にペットに関する記述、各列にその属性が含まれています。この情報を使用して、ペットが引き取り可能であるかどうかを予測します。

以下は、このデータセットの説明です。数値とカテゴリカルのカラムがあることに注意してください。自由テキストのカラムもありアンスが、このチュートリアルでは使用しません。

カラム | 説明 | 特徴量タイプ | データ型
--- | --- | --- | ---
Type | 動物の種類（犬、猫） | カテゴリカル | 文字列
Age | ペットの年齢 | 数値 | 整数
Breed1 | ペットの主な品種 | カテゴリカル | 文字列
Color1 | ペットの毛色 1 | カテゴリカル | 文字列
Color2 | ペットの毛色 2 | カテゴリカル | 文字列
MaturitySize | 成獣時のサイズ | カテゴリカル | 文字列
FurLength | 毛の長さ | カテゴリカル | 文字列
Vaccinated | 予防接種済み | カテゴリカル | 文字列
Sterilized | 不妊手術済み | カテゴリカル | 文字列
Health | 健康状態 | カテゴリカル | 文字列
Fee | 引き取り料 | 数値 | 整数
Description | ペットのプロフィール | テキスト | 文字列
PhotoAmt | アップロードされたペットの写真数 | 数値 | 整数
AdoptionSpeed | 引き取りまでの期間 | 分類 | 整数

## TensorFlow とその他のライブラリをインポートする


In [ ]:
!pip install -q sklearn

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
tf.__version__

## Pandas を使用してデータフレームを作成する

[Pandas](https://pandas.pydata.org/) は、構造化データの読み込みと処理を支援するユーティリティが多数含まれる Python ライブラリです。Pandas を使用し、URL からデータセットをダウンロードしてデータフレームに読み込みます。

In [ ]:
import pathlib

dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

In [ ]:
dataframe.head()

## ターゲット変数を作成する

Kaggle コンペティションでは、ペットが引き取られるまでの期間（1 週目、1 か月目、3 か月目など）を予測することがタスクとなっていますが、このチュートリアルでは、このタスクを単純化しましょう。ここでは、このタスクを二項分類問題にし、単にペットが引き取られるかどうかのみを予測します。

ラベルカラムを変更すると、0 は引き取られなかった、1 は引き取られたことを示すようになります。

In [ ]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

## データフレームを train、validation、および test に分割する

ダウンロードしたデータセットは単純な CSV ファイルです。これを train、validation、および test セットに分割します。

In [ ]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

## tf.data を使用して入力パイプラインを作成する

次に、データをシャッフルしてバッチ化するために、データフレームを [tf.data](https://www.tensorflow.org/guide/datasets) でラップします。非常に大型（メモリに収まらないほどの規模）の CSV ファイルを処理している場合は、tf.data を使用してディスクから直接読み取ります。この方法は、このチュートリアルでは説明していません。

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

入力パイプラインを作成したので、それを呼び出して、戻されるデータのフォーマットを確認しましょう。出力の可読性を維持するために、小さなバッチを使用しました。

In [ ]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

In [ ]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of targets:', label_batch )

ご覧のとおり、データセットは、データフレームの行からカラムの値にマップしているカラム名の（データフレームのカラム名）のディクショナリを返しています。

## 前処理レイヤーの使用を実演する

Keras Preprocessing Layers API を使うと、Keras ネイティブの入力処理パイプラインを構築することができます。特徴量の前処理コードを実演するために、3 つの前処理レイヤーを使用します。

- [`Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Normalization) - データの特徴量方向の正規化。
- [`CategoryEncoding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/CategoryEncoding) - カテゴリのエンコーディングレイヤー。
- [`StringLookup`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/StringLookup) - ボキャブラリから整数インデックスに文字列をマッピングします。
- [`IntegerLookup`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/IntegerLookup) - ボキャブラリから整数インデックスに整数をマッピングします。

使用できる前処理レイヤーのリストは、[こちら](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing)をご覧ください。

### 数値カラム

数値特徴量ごとに、各特徴量の平均が 0、標準偏差が 1 となるように Normalization() レイヤーを使用します。

`get_normalization_layer` 関数は、特徴量方向の正規化を数値特徴量に適用するレイヤーを返します。

In [ ]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization(axis=None)

  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [ ]:
photo_count_col = train_features['PhotoAmt']
layer = get_normalization_layer('PhotoAmt', train_ds)
layer(photo_count_col)

注意: 多数の特徴量（数百個以上）がある場合は、先にそれらを連結してから単一の [normalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Normalization) レイヤーを使用するとより効率的です。

### カテゴリカルカラム

このデータセットでは、Type は文字列として表現されています（'Dog' または 'Cat'）。文字列を直接モデルに注入することはできないため、前処理レイヤーを使って文字列をワンホットベクトルとして表現します。

`get_category_encoding_layer` 関数は、ボキャブラリの値を整数インデックスにマッピングして特徴量をワンホットエンコーディングするレイヤーを返します。

In [ ]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_tokens=max_tokens)

  # Prepare a Dataset that only yields our feature
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply one-hot encoding to our indices. The lambda function captures the
  # layer so we can use them, or include them in the functional model later.
  return lambda feature: encoder(index(feature))

In [ ]:
type_col = train_features['Type']
layer = get_category_encoding_layer('Type', train_ds, 'string')
layer(type_col)

数値を直接モデルに注入せずに、それらの入力のワンホットエンコーディングを使用することがよくあります。ペットの年齢を表す未加工のデータを考察しましょう。

In [ ]:
type_col = train_features['Age']
category_encoding_layer = get_category_encoding_layer('Age', train_ds,
                                                      'int64', 5)
category_encoding_layer(type_col)

## 使用するカラムを選択する

さまざまな種類の前処理レイヤーが使用される様子を見てきましたが、今後は、それを使ってモデルをトレーニングする方法を見てみましょう。[Keras-functional API](https://www.tensorflow.org/guide/keras/functional) を使用して、モデルを構築します。Keras functional API は、[tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) API より柔軟なモデルを作成するのに適しています。

このチュートリアルでは、前処理レイヤーを使用するために必要な全コード（mechanic）などを示すことを目的としています。モデルをトレーニングするためにいくつかのカラムが任意に選択されています。

重要ポイント: 正確なモデルの構築を目的としている場合は、より大きなデータセットを独自に用意し、どの特徴量を含めるのが最も意義が高く、どのように表現すrべきかについてよく考えましょう。

最初の方で、入力パイプラインを実演するために小さなバッチを使用しました。今度はより大きなバッチサイズで新しい入力パイプラインを作成してみましょう。


In [ ]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
all_inputs = []
encoded_features = []

# Numeric features.
for header in ['PhotoAmt', 'Fee']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

In [ ]:
# Categorical features encoded as integers.
age_col = tf.keras.Input(shape=(1,), name='Age', dtype='int64')
encoding_layer = get_category_encoding_layer('Age', train_ds, dtype='int64',
                                             max_tokens=5)
encoded_age_col = encoding_layer(age_col)
all_inputs.append(age_col)
encoded_features.append(encoded_age_col)

In [ ]:
# Categorical features encoded as string.
categorical_cols = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                    'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Breed1']
for header in categorical_cols:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string',
                                               max_tokens=5)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)


## モデルを作成、コンパイル、およびトレーニングする


エンドツーエンドのモデルを作成できるようになりました。

In [ ]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

接続性グラフを視覚化しましょう。


In [ ]:
# rankdir='LR' is used to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")


### モデルをトレーニングする


In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

## 新しいデータの推論

重要ポイント: 開発したモデルには前処理コードが含まれているため、直接 CSV ファイルから行を分類できるようになりました。


Keras モデルを保存して、再読み込みすることができます。TensorFlow モデルの詳細については、[こちら](https://www.tensorflow.org/tutorials/keras/save_and_load)のチュートリアルをご覧ください。

In [ ]:
model.save('my_pet_classifier')
reloaded_model = tf.keras.models.load_model('my_pet_classifier')

`model.predict()` を呼び出すだけで、新しいサンプルの予測を得ることができます。以下の 2 つを行ってください。

1. バッチに次元をを持たせるために、スカラーをリストにラップします（モデルは、単一のサンプルではなく、データのバッチのみを処理します）。
2. 各特徴量で `convert_to_tensor` を呼び出します。

In [ ]:
sample = {
    'Type': 'Cat',
    'Age': 3,
    'Breed1': 'Tabby',
    'Gender': 'Male',
    'Color1': 'Black',
    'Color2': 'White',
    'MaturitySize': 'Small',
    'FurLength': 'Short',
    'Vaccinated': 'No',
    'Sterilized': 'No',
    'Health': 'Healthy',
    'Fee': 100,
    'PhotoAmt': 2,
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(
    "This particular pet had a %.1f percent probability "
    "of getting adopted." % (100 * prob)
)

重要ポイント: 通常、データベースの規模が大きく複雑であるほど、ディープラーニングの結果がよくなります。このチュートリアルのデータセットのように、小さなデータセットを使用する場合は、決定木またはランダムフォレストを強力なベースラインとして使用することをお勧めします。このチュートリアルでは、構造化データとの連携の仕組みを実演することが目的であるため、コードは将来的に独自のデータセットを使用する際の出発点として使用することができます。

## 次のステップ

構造化データの分類をさらに学習するには、自分で試すのが最善です。別のデータセットを使用し、上記に似たコードを使用し、モデルのトレーニングと分類をおこなうと良いでしょう。精度を改善するには、モデルに含める特徴量とその表現方法を吟味ししてください。